# Music Generator
## COMP 432 Fall 2020
## Natalia Whiteley 40044353 and Michael Naccache 27...

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from music21 import *

In [5]:
# instruments to include from the midi file
INSTRUMENTS = [instrument.Piano, instrument.Bass, instrument.Guitar, instrument.Saxophone, instrument.Trumpet]
VERBOSE = True

## Part 1: Import Dataset

Helper functions for parsing midi files and extracting files from the folders

In [6]:
# given the name of a midi file, returns a 2D array of notes per instrument
def import_midi(midi_file, verbose=VERBOSE):

    # parse midi file
    midi = converter.parse(midi_file)
  
    # seperate based on instrument
    midi_parts = instrument.partitionByInstrument(midi)

    notes = {}
    notes_to_parse = []

    for part in midi_parts.parts:
        # only use instruments we want
        if any(isinstance(part.getInstrument(), x) for x in INSTRUMENTS):
            notes[str(part.getInstrument().bestName())] = []
            notes_to_parse = part.recurse()

            # finding whether a particular element is note or a chord
            for element in notes_to_parse: 
                #note
                if isinstance(element, note.Note):
                    notes[str(part.getInstrument().bestName())].append(str(element.pitch))
                
                #chord
                elif isinstance(element, chord.Chord):
                    notes[str(part.getInstrument().bestName())].append('.'.join(str(n) for n in element.normalOrder))

    # remove any empty instruments
    notes = {k: np.array(v) for k, v in notes.items() if v}

    if verbose:
        print('Finished importing {}.'.format(midi_file))

    return notes

# gets the path and label for every file in a directory and its sub-directories
def get_all_files_in_dir(dir_path, verbose=VERBOSE):
    midis = []
    genres = []
    for root, dirs, files in os.walk(dir_path):
        genre = os.path.basename(root)

        for file in files:
            if file.endswith('.mid'):
                midis.append(os.path.join(root, file))
                genres.append(genre)
        
        if verbose:
            print('Loaded all {} midi files.'.format(genre))

    return midis, genres


In [7]:
all_midis, genres = get_all_files_in_dir('./midi_files/original')
all_midis = [import_midi(x) for x in all_midis]

print(all_midis[0])
print(genres[0])

Loaded all original midi files.
Finished importing ./midi_files/original/small planet (original)_ timo raita.mid.
Finished importing ./midi_files/original/high_high_over_the_mountains_jt2.mid.
Finished importing ./midi_files/original/always_you-xg_dw.mid.
Finished importing ./midi_files/original/Midnight Blues_dw.mid.
Finished importing ./midi_files/original/love waves_ah.mid.
Finished importing ./midi_files/original/jaberwocky_fiz.mid.
Finished importing ./midi_files/original/say_what_you_want-kar_dw.mid.
Finished importing ./midi_files/original/changes_dw.mid.
Finished importing ./midi_files/original/houseproud mrs petty_ah.mid.
Finished importing ./midi_files/original/brians_theme_fiz.mid.
Finished importing ./midi_files/original/jazz_heads_dw.mid.
Finished importing ./midi_files/original/time_on_our_hands_in _A_bz2_bz3.mid.
Finished importing ./midi_files/original/the sunshine in your eyes_ah.mid.
Finished importing ./midi_files/original/as far as you can see_ sascha henken.mid.
Fi

## Part 2: Data Analysis

In [ ]:
# graph distribution of all images

# graph distribution of train images

# graph distribution of test images

## Part 3: Normalize Dataset

In [ ]:
# convert notes to ints

## Part 3: Train GAN With Magenta/Wavenet

## Part 4: Train TensorFlow Models

## Part 5: Compare Results

## Sources

https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/